Task 2: Develop Time Series Forecasting Models for All Datasets

In [6]:
import pandas as pd
import itertools
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

datasets = {
    'TSLA': pd.read_csv('../data/TSLA_handler_update.csv', parse_dates=['Date'], index_col='Date'),
    'BND': pd.read_csv('../data/BND_handler_update.csv', parse_dates=['Date'], index_col='Date'),
    'SPY': pd.read_csv('../data/SPY_handler_update.csv', parse_dates=['Date'], index_col='Date')
}

In [7]:
train_size_ratio = 0.8
for name, df in datasets.items():
    train_size = int(len(df) * train_size_ratio)
    train, test = df[:train_size], df[train_size:]

### ARIMA Model

In [10]:
from itertools import product
import warnings
warnings.filterwarnings("ignore")

pdq = list(itertools.product(range(0, 3), range(0, 3), range(0, 3))) 

for name, df in datasets.items():

    train, test = df[:train_size], df[train_size:]
    
    best_score, best_params = float("inf"), None
    for param in pdq:
        try:
            model = ARIMA(train['Close'], order=param)
            model_fit = model.fit()
            predictions = model_fit.forecast(steps=len(test))
            score = mean_absolute_error(test['Close'], predictions)
            if score < best_score:
                best_score, best_params = score, param
        except:
            continue
    
    

    best_model = ARIMA(train['Close'], order=best_params)
    best_model_fit = best_model.fit()
    

    predictions = best_model_fit.forecast(steps=len(test))
    mae = mean_absolute_error(test['Close'], predictions)
    rmse = mean_squared_error(test['Close'], predictions, squared=False)
    mape = (np.abs((test['Close'] - predictions) / test['Close']).mean()) * 100
    
    print(f"{name} - ARIMA - MAE: {mae}, RMSE: {rmse}, MAPE: {mape}%")


TSLA - ARIMA - MAE: 55.40882176163719, RMSE: 73.95427027085168, MAPE: nan%
BND - ARIMA - MAE: 2.2008995531770585, RMSE: 2.5375704873241, MAPE: nan%
SPY - ARIMA - MAE: 25.368042064700624, RMSE: 30.42663058773151, MAPE: nan%
